# 데이터셋 정제: 키워드 추상화

기존 `training_dataset.jsonl`의 Input 키워드를 형태소 분석을 통해 핵심 명사만 추출하여 추상화합니다.
이를 통해 모델이 문장을 짜깁기하는 대신 새로운 문장을 생성하도록 개선합니다.


## ⚙️ 사전 조건
- `training_dataset.jsonl` 파일 존재
- Colab T4 이상 GPU 할당 (형태소 분석용)


---
## 0. 필수 패키지 설치


In [1]:
%pip install -q konlpy

print("✅ KoNLPy 설치 완료")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 40.2 MB/s eta 0:00:00
✅ KoNLPy 설치 완료


In [2]:
from google.colab import drive

# Google Drive 마운트 (이미 마운트되어 있다면 건너뛰어도 됩니다)
drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")


Mounted at /content/drive
✅ Google Drive 마운트 완료


---
## 1. 경로 설정 및 데이터 로드


In [3]:
from pathlib import Path
import json
import re

PROJECT_ROOT = Path("/content/drive/MyDrive//board_crawling")
OUTPUT_DIR = PROJECT_ROOT / "outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_JSONL = OUTPUT_DIR / "training_dataset.jsonl"
REFINED_JSONL = OUTPUT_DIR / "training_dataset_refined.jsonl"

if not TRAIN_JSONL.exists():
    raise FileNotFoundError(f"{TRAIN_JSONL}가 없습니다.")

training_data = []
with open(TRAIN_JSONL, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            training_data.append(json.loads(line))

print(f"✅ 원본 데이터셋 로드 완료: {len(training_data)}개 레코드")


✅ 원본 데이터셋 로드 완료: 6591개 레코드


---
## 2. 형태소 분석기 초기화


In [4]:
from konlpy.tag import Okt

okt = Okt()
print("✅ 형태소 분석기(Okt) 초기화 완료")


✅ 형태소 분석기(Okt) 초기화 완료


---
## 3. 키워드 추출 및 정제 함수


In [5]:
def extract_keywords_from_text(text, prioritize_nouns=True):
    """
    텍스트에서 명사, 동사, 형용사를 추출합니다.
    prioritize_nouns=True이면 명사를 우선적으로 포함합니다.
    """
    if not text or text.strip() == "":
        return []

    try:
        morphs = okt.pos(text, norm=True, stem=True)

        # 명사 우선 추출
        nouns = [word for word, pos in morphs if pos == 'Noun']
        # 동사, 형용사 추출
        verbs_adjs = [word for word, pos in morphs if pos in ['Verb', 'Adjective']]

        if prioritize_nouns:
            # 명사 우선, 동사/형용사는 보조적으로
            keywords = nouns + verbs_adjs[:2]  # 동사/형용사는 최대 2개
        else:
            keywords = nouns + verbs_adjs

        return keywords
    except:
        return []

def refine_keywords(input_text, output_text):
    """
    Input의 키워드를 추상화하고, Output에서 핵심 키워드(명사, 동사, 형용사)를 추출하여 결합합니다.
    """
    # Input에서 "키워드: " 부분 제거
    if input_text.startswith("키워드:"):
        keyword_str = input_text[len("키워드:"):].strip()
        original_keywords = [kw.strip() for kw in keyword_str.split(',') if kw.strip()]
    else:
        original_keywords = []

    # nan 제거
    original_keywords = [kw for kw in original_keywords if kw.lower() != 'nan' and kw.strip() != '']

    # Output 텍스트에서 키워드 추출 (명사, 동사, 형용사)
    output_keywords = extract_keywords_from_text(output_text)

    # Input 키워드에서도 키워드 추출
    input_keywords = []
    for kw in original_keywords:
        keywords = extract_keywords_from_text(kw)
        input_keywords.extend(keywords)

    # 모든 키워드 결합 및 중복 제거
    all_keywords = list(dict.fromkeys(input_keywords + output_keywords))  # 순서 유지하면서 중복 제거

    # 핵심 키워드 3~5개 선별
    refined_keywords = all_keywords[:5]

    # 최소 3개는 확보 (부족하면 원본 키워드에서 의미있는 것 추가)
    if len(refined_keywords) < 3 and original_keywords:
        # 원본 키워드 중 의미있는 것 추가
        for kw in original_keywords:
            if kw not in refined_keywords and len(kw) > 1:  # 너무 짧은 것 제외
                refined_keywords.append(kw)
                if len(refined_keywords) >= 5:
                    break

    return refined_keywords[:5]  # 최대 5개

def is_valid_data(entry):
    """
    nan이 포함된 데이터를 필터링합니다.
    """
    input_text = entry.get("input", "")
    output_text = entry.get("output", "")

    # input이나 output에 nan이 포함되어 있으면 무조건 제외
    if "nan" in input_text.lower() or "nan" in output_text.lower():
        return False

    # output이 비어있거나 너무 짧으면 제외
    if not output_text or len(output_text.strip()) < 10:
        return False

    return True


print("✅ 키워드 정제 함수 정의 완료")


✅ 키워드 정제 함수 정의 완료


---
## 4. 데이터셋 정제 실행


In [6]:
refined_data = []
skipped_count = 0
nan_removed_count = 0

for idx, entry in enumerate(training_data):
    # nan 데이터 필터링
    if not is_valid_data(entry):
        nan_removed_count += 1
        continue

    instruction = entry.get("instruction", "")
    input_text = entry.get("input", "")
    output_text = entry.get("output", "")

    # 키워드 추상화
    try:
        refined_keywords = refine_keywords(input_text, output_text)

        # 키워드가 너무 적으면 스킵 (최소 2개)
        if len(refined_keywords) < 2:
            skipped_count += 1
            continue

        # 새로운 input 생성
        new_input = f"키워드: {', '.join(refined_keywords)}"

        # 정제된 데이터 생성
        refined_entry = {
            "instruction": instruction,
            "input": new_input,
            "output": output_text
        }
        refined_data.append(refined_entry)

    except Exception as e:
        print(f"⚠️ 레코드 {idx} 처리 중 오류: {e}")
        skipped_count += 1
        continue

    # 진행 상황 출력 (100개마다)
    if (idx + 1) % 100 == 0:
        print(f"진행 중: {idx + 1}/{len(training_data)} 처리 완료")

print(f"\n✅ 데이터셋 정제 완료")
print(f"   - 원본: {len(training_data)}개")
print(f"   - 정제 후: {len(refined_data)}개")
print(f"   - nan 제거: {nan_removed_count}개")
print(f"   - 기타 스킵: {skipped_count - nan_removed_count}개")


진행 중: 100/6591 처리 완료
진행 중: 200/6591 처리 완료
진행 중: 300/6591 처리 완료
진행 중: 500/6591 처리 완료
진행 중: 600/6591 처리 완료
진행 중: 700/6591 처리 완료
진행 중: 800/6591 처리 완료
진행 중: 900/6591 처리 완료
진행 중: 1000/6591 처리 완료
진행 중: 1100/6591 처리 완료
진행 중: 1200/6591 처리 완료
진행 중: 1300/6591 처리 완료
진행 중: 1500/6591 처리 완료
진행 중: 1600/6591 처리 완료
진행 중: 1700/6591 처리 완료
진행 중: 1800/6591 처리 완료
진행 중: 1900/6591 처리 완료
진행 중: 2000/6591 처리 완료
진행 중: 2100/6591 처리 완료
진행 중: 2200/6591 처리 완료
진행 중: 2300/6591 처리 완료
진행 중: 2400/6591 처리 완료
진행 중: 2500/6591 처리 완료
진행 중: 2600/6591 처리 완료
진행 중: 2800/6591 처리 완료
진행 중: 2900/6591 처리 완료
진행 중: 3000/6591 처리 완료
진행 중: 3100/6591 처리 완료
진행 중: 3200/6591 처리 완료
진행 중: 3300/6591 처리 완료
진행 중: 3400/6591 처리 완료
진행 중: 3500/6591 처리 완료
진행 중: 3600/6591 처리 완료
진행 중: 3700/6591 처리 완료
진행 중: 3800/6591 처리 완료
진행 중: 3900/6591 처리 완료
진행 중: 4000/6591 처리 완료
진행 중: 4100/6591 처리 완료
진행 중: 4200/6591 처리 완료
진행 중: 4600/6591 처리 완료
진행 중: 4800/6591 처리 완료
진행 중: 4900/6591 처리 완료
진행 중: 5000/6591 처리 완료
진행 중: 5100/6591 처리 완료
진행 중: 5200/6591 처리 완료
진행 중: 5300/6591 처리

In [7]:
# 정제된 데이터셋 저장
with open(REFINED_JSONL, "w", encoding="utf-8") as f:
    for entry in refined_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"✅ 정제된 데이터셋 저장 완료: {REFINED_JSONL}")
print(f"   - 총 {len(refined_data)}개 레코드 저장됨")


✅ 정제된 데이터셋 저장 완료: /content/drive/MyDrive/board_crawling/outputs/training_dataset_refined.jsonl
   - 총 5752개 레코드 저장됨


---
## 6. 샘플 확인


In [8]:
# 샘플 몇 개 확인
print("=== 정제 전후 비교 샘플 ===\n")

# 원본 데이터에서 샘플 찾기
sample_indices = [1, 2, 5]  # 몇 개 샘플 확인

for sample_idx in sample_indices[:3]:
    if sample_idx < len(training_data):
        original = training_data[sample_idx]
        # 정제된 데이터에서 같은 output을 가진 것 찾기 (output이 고유함)
        refined_sample = None
        for ref in refined_data:
            if ref["output"] == original["output"]:  # output으로 매칭
                refined_sample = ref
                break

        if refined_sample:
            print(f"[샘플 {sample_idx}]")
            print(f"원본 Input: {original['input']}")
            print(f"정제 Input: {refined_sample['input']}")
            print(f"Output: {refined_sample['output'][:100]}...")
            print()
        else:
            print(f"[샘플 {sample_idx}] - 정제 과정에서 스킵됨")
            print()

=== 정제 전후 비교 샘플 ===

[샘플 1]
원본 Input: 키워드: 남친은, 근데, 나, 잘생겼는데, 나는, 객관적으로, 잘생겼어, 훈훈
정제 Input: 키워드: 남친, 나, 잘생기다, 객관, 훈훈
Output: 남친은 잘생겼는데 나는..

남친은 객관적으로 잘생겼어... 훈훈...근데 난 아닌 것 같거든??근데 남친은 맨날 나 너무 예쁘다고 해줘...나 거울 볼 때마다 우울해ㅠㅠㅠㅠㅠㅠㅠ...

[샘플 2]
원본 Input: 키워드: 가츠벤또, 제치고, 밥약, 1티어, 등극한, 식당, 인정하면, 개추
정제 Input: 키워드: 가츠, 벤또, 제, 밥약, 티어
Output: 가츠벤또 제치고 밥약 1티어 등극한 식당..

인정하면 개추 ㅇㅅㅇ...

[샘플 5] - 정제 과정에서 스킵됨

